## **Naive Bayes Algorithm on adult dataset**

In [1]:
import numpy as np
import pandas as pd
import random
import sys
from random import randrange
from math import sqrt
from math import pi
from math import exp
from math import isnan

In [2]:
# Stratifying the data into training data set and testing data set
# The testSize is specified as ratio

def trainTestSplit(data, testSize):
    # Converting the test size (ratio) into numbers of rows 
    if isinstance(testSize, float):
        testSize = round(testSize * len(data))
        
    # Getting the index of the data from the dataset
    dataIndex = data.index.tolist()
    
    # Generate random indexes to split the dataset
    testIndex = random.sample(population = dataIndex, k = testSize)
    
    # Allocate the testing set according to the random indexes (testIndex)
    testSet = data.loc[testIndex]
    
    # Dropping the testset rows from the original data set, and make the remaining rows as the training set
    trainSet = data.drop(testIndex)
    
    # Return the train and test set data
    return trainSet, testSet

In [3]:
# Calculate the Gaussian probability distribution for continuous features
# To prevent numerical underflow, when the exponent underflows is out of Pythons' float precision, set
# exponent to the smallest possible float value by the system
# To prevent the zero frequency/count problem, the function catches occurences of 0/null standard deviation
# and returns the probability of 1/number of observations

def calculateGaussianProbability(x, mean, stdev, totalRows):
    if stdev == 0 or isnan(stdev):
        return 1/totalRows # if stdev is 0, return the probability of "Add one count"
    
    exponent = exp(-((x - mean)**2 / (2 * stdev**2 )))
    
    if exponent == 0:
        # Set exponent to smallest possible float supported by the system
        exponent = sys.float_info.min
        
    return (1 / (sqrt(2 * pi) * stdev)) * exponent

In [4]:
# Calculate probability for categorical features
# To prevent the zero frequency/count problem, the function catches the occurences of 0/null standard deviation

def calculateProbability(x, X1, count1, X2, count2, classCount, totalRows):
    if x == X1:
        # If zero frequency occurs, add 1 to count and return the probability
        if count1/classCount == 0:
            return 1/totalRows
        
        return count1/classCount
    else:
        # If zero frequency occurs, add 1 to count and return the probability
        if count2/classCount == 0:
            return 1/totalRows
        
        return count2/classCount

In [5]:
# Calculate the probabilities of predicting each class for a given row:
# For continuous features use Gaussian probability function
# For categorical feature calculate_probability function

def calculateClassProbabilities(summaries, row):
    
    # Get the length of the dataset
    # Sum up all the counts of each label class
    totalRows = sum([summaries[label][0][2] for label in summaries])
    
    # Instantiate a dictionary to store probability of each label class for a given row
    probabilities = dict()
    
    # Get the class value: classValue
    # Get the summaries for each class: classSummaries
    for classValue, classSummaries in summaries.items():
        
        # Get the probability of each label class e.g., If class label 1 has a length of 12345
        # and length of dataset is 234567 then this probability is 12345/234567
        probabilities[classValue] = summaries[classValue][0][2]/float(totalRows)
        
        #looping through each summaries
        for i in range(len(classSummaries)):
            # if categorical feature
            if len(classSummaries[i]) > 3:
                X1, count1, X2, count2, classCount = classSummaries[i]
                probabilities[classValue] = \
                probabilities[classValue] * calculateProbability(row[i], X1, count1, X2, count2, classCount, totalRows)
            # if continuous feature
            else:
                mean, stdev, _ = classSummaries[i]
                probabilities[classValue] = \
                probabilities[classValue] * calculateGaussianProbability(row[i], mean, stdev, totalRows)
                
    return probabilities

In [6]:
# Predict the class for a given row
def predictClass(summaries, row):
    # Storing the probabilities by calling the method
    probabilities = calculateClassProbabilities(summaries, row)
    
    # Initializing the variables
    bestLabel, bestProb = None, -1
    
    # Looping through to find the best label with the best probability
    for classVal, probability in probabilities.items():
        if bestLabel is None or probability > bestProb:
            bestProb = probability
            bestLabel = classVal
    return bestLabel

In [7]:
def summarizeDataset(data):
    # Creating a empty dict to store the target class
    summaries = {}
    # For loop to loop the target column that are unique
    for i in data.iloc[:,-1].unique():
        # listing the feature variable of each unique class
        feature = []
        # looping through all the feature variable except the target column
        for j in range(len(data.columns)-1):
            # Storing the size of unique values in a variable
            uniqueValueSize = len(data.iloc[:,j].unique())
            
            # If categorical feature
            if(uniqueValueSize < 5):
                aList = list()
                # Creating a subset for each class
                df = data[data.iloc[:,-1] == i]
                
                # looping through the categorical feature for unique vals
                for k in data.iloc[:,j].unique():
                    # storing the unique value and counting the val
                    aList.append(k)
                    aList.append(len(df[df.iloc[:,j] == k]))
                feature.append(tuple([aList[0], aList[1], aList[2], aList[3], len(data[data.iloc[:,-1] == i])]))
            # else continuous feature
            else:
                feature.append((data[data.iloc[:,-1] == i].mean(axis = 0)[j], \
                                data[data.iloc[:,-1] == i].std(axis = 0)[j], len(data[data.iloc[:,-1] == i])))
        # storing the summaries     
        summaries[i] = feature
    return summaries

In [8]:
# Naive Bayes Classifier Algorithm to predict the train set and the test set
def naiveBayesian(trainSet, testSet):
    # Method to get the summary
    summary = summarizeDataset(trainSet)
    
    # Creating an empty list
    predictions = list()
    
    # looping through the test set values 
    for row in testSet.values:
        output = predictClass(summary, row)
        predictions.append(output)
        
    return(predictions)

In [9]:
# Calculated as:
# check for equality of predicted value and labels in test_set
# calculates the sum of correct prediction
# divides the sum by length of test_set

    
import numpy as np
from math import sqrt

def accuracy(predictions, dataSet):
    yTest = list(dataSet.iloc[:, -1])
    assert len(predictions) == len(yTest), "Predictions and actual values have different lengths"
    
    correctCount = 0
    sumError = 0.0
    rsmeError = 0.0
    for i in range(len(yTest)):
        if predictions[i] == yTest[i]:
            correctCount += 1
        sumError += abs(predictions[i] - yTest[i])
        rsmeError += (predictions[i] - yTest[i]) ** 2
        
    print(f'Number of exact matches in predictions: {correctCount}/{len(yTest)}')        
    print(f'Mean Squared Error(MSE): {np.square(np.subtract(yTest, predictions)).mean()}')
    print(f'Root Mean Squared Error (RMSE): {sqrt(rsmeError/float(len(yTest)))}')
    print(f'Mean Absolute Error(MSE): {sumError/float(len(yTest))}')
    print(f'Length of predictions: {len(predictions)}')
    print(f'Length of actual values: {len(yTest)}')
    
    return (round(correctCount / len(dataSet) * 100, 3))





In [10]:
colNames = ['age', 'workclass', 'fnlwgt', 'education', 'education_num', 'marital_status', 'occupation', 'relationship',
             'race', 'sex', 'capital_gain', 'capital_loss', 'hours_per_week', 'native_country', 'income']

df = pd.read_csv('adult.data', sep=',', names = colNames, engine='python')

df.head()

age          workclass  fnlwgt   education  education_num  \
0   39          State-gov   77516   Bachelors             13   
1   50   Self-emp-not-inc   83311   Bachelors             13   
2   38            Private  215646     HS-grad              9   
3   53            Private  234721        11th              7   
4   28            Private  338409   Bachelors             13   

        marital_status          occupation    relationship    race      sex  \
0        Never-married        Adm-clerical   Not-in-family   White     Male   
1   Married-civ-spouse     Exec-managerial         Husband   White     Male   
2             Divorced   Handlers-cleaners   Not-in-family   White     Male   
3   Married-civ-spouse   Handlers-cleaners         Husband   Black     Male   
4   Married-civ-spouse      Prof-specialty            Wife   Black   Female   

   capital_gain  capital_loss  hours_per_week  native_country  income  
0          2174             0              40   United-States   <=50K  
1             0             0              13   United-States   <=50K  
2             0             0              40   United-States   <=50K  
3             0             0              40   United-States   <=50K  
4             0             0              40            Cuba   <=50K

In [11]:
df.head().info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             5 non-null      int64 
 1   workclass       5 non-null      object
 2   fnlwgt          5 non-null      int64 
 3   education       5 non-null      object
 4   education_num   5 non-null      int64 
 5   marital_status  5 non-null      object
 6   occupation      5 non-null      object
 7   relationship    5 non-null      object
 8   race            5 non-null      object
 9   sex             5 non-null      object
 10  capital_gain    5 non-null      int64 
 11  capital_loss    5 non-null      int64 
 12  hours_per_week  5 non-null      int64 
 13  native_country  5 non-null      object
 14  income          5 non-null      object
dtypes: int64(6), object(9)
memory usage: 728.0+ bytes


In [12]:
#encode the object attributes
newdf = pd.Series(df['workclass'], dtype = "category")
df['workclass'] = newdf.cat.codes

newdf = pd.Series(df['education'], dtype = "category")
df['education'] = newdf.cat.codes

newdf = pd.Series(df['marital_status'], dtype = "category")
df['marital_status'] = newdf.cat.codes

newdf = pd.Series(df['occupation'], dtype = "category")
df['occupation'] = newdf.cat.codes

newdf = pd.Series(df['relationship'], dtype = "category")
df['relationship'] = newdf.cat.codes

newdf = pd.Series(df['race'], dtype = "category")
df['race'] = newdf.cat.codes

newdf = pd.Series(df['sex'], dtype = "category")
df['sex'] = newdf.cat.codes

newdf = pd.Series(df['native_country'], dtype = "category")
df['native_country'] = newdf.cat.codes

newdf = pd.Series(df['income'], dtype = "category")
df['income'] = newdf.cat.codes

In [13]:
df.describe()

age     workclass        fnlwgt     education  education_num  \
count  32561.000000  32561.000000  3.256100e+04  32561.000000   32561.000000   
mean      38.581647      3.868892  1.897784e+05     10.298210      10.080679   
std       13.640433      1.455960  1.055500e+05      3.870264       2.572720   
min       17.000000      0.000000  1.228500e+04      0.000000       1.000000   
25%       28.000000      4.000000  1.178270e+05      9.000000       9.000000   
50%       37.000000      4.000000  1.783560e+05     11.000000      10.000000   
75%       48.000000      4.000000  2.370510e+05     12.000000      12.000000   
max       90.000000      8.000000  1.484705e+06     15.000000      16.000000   

       marital_status    occupation  relationship          race           sex  \
count    32561.000000  32561.000000  32561.000000  32561.000000  32561.000000   
mean         2.611836      6.572740      1.446362      3.665858      0.669205   
std          1.506222      4.228857      1.606771      0.848806      0.470506   
min          0.000000      0.000000      0.000000      0.000000      0.000000   
25%          2.000000      3.000000      0.000000      4.000000      0.000000   
50%          2.000000      7.000000      1.000000      4.000000      1.000000   
75%          4.000000     10.000000      3.000000      4.000000      1.000000   
max          6.000000     14.000000      5.000000      4.000000      1.000000   

       capital_gain  capital_loss  hours_per_week  native_country  \
count  32561.000000  32561.000000    32561.000000    32561.000000   
mean    1077.648844     87.303830       40.437456       36.718866   
std     7385.292085    402.960219       12.347429        7.823782   
min        0.000000      0.000000        1.000000        0.000000   
25%        0.000000      0.000000       40.000000       39.000000   
50%        0.000000      0.000000       40.000000       39.000000   
75%        0.000000      0.000000       45.000000       39.000000   
max    99999.000000   4356.000000       99.000000       41.000000   

             income  
count  32561.000000  
mean       0.240810  
std        0.427581  
min        0.000000  
25%        0.000000  
50%        0.000000  
75%        0.000000  
max        1.000000

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   age             32561 non-null  int64
 1   workclass       32561 non-null  int8 
 2   fnlwgt          32561 non-null  int64
 3   education       32561 non-null  int8 
 4   education_num   32561 non-null  int64
 5   marital_status  32561 non-null  int8 
 6   occupation      32561 non-null  int8 
 7   relationship    32561 non-null  int8 
 8   race            32561 non-null  int8 
 9   sex             32561 non-null  int8 
 10  capital_gain    32561 non-null  int64
 11  capital_loss    32561 non-null  int64
 12  hours_per_week  32561 non-null  int64
 13  native_country  32561 non-null  int8 
 14  income          32561 non-null  int8 
dtypes: int64(6), int8(9)
memory usage: 1.8 MB


In [15]:
# Split the dataset into training and tetrainSet, testSet = trainTestSplit(df, 0.3)sting
trainSet, testSet = trainTestSplit(df, 0.3)
trainSet.describe()

age     workclass        fnlwgt     education  education_num  \
count  22793.000000  22793.000000  2.279300e+04  22793.000000   22793.000000   
mean      38.585662      3.877111  1.901105e+05     10.306805      10.080814   
std       13.652932      1.450070  1.058585e+05      3.868242       2.577741   
min       17.000000      0.000000  1.376900e+04      0.000000       1.000000   
25%       28.000000      4.000000  1.180230e+05      9.000000       9.000000   
50%       37.000000      4.000000  1.784700e+05     11.000000      10.000000   
75%       48.000000      4.000000  2.368790e+05     12.000000      12.000000   
max       90.000000      8.000000  1.484705e+06     15.000000      16.000000   

       marital_status    occupation  relationship          race           sex  \
count    22793.000000  22793.000000  22793.000000  22793.000000  22793.000000   
mean         2.612732      6.583600      1.452507      3.661870      0.666169   
std          1.511750      4.239418      1.606067      0.852624      0.471590   
min          0.000000      0.000000      0.000000      0.000000      0.000000   
25%          2.000000      3.000000      0.000000      4.000000      0.000000   
50%          2.000000      7.000000      1.000000      4.000000      1.000000   
75%          4.000000     10.000000      3.000000      4.000000      1.000000   
max          6.000000     14.000000      5.000000      4.000000      1.000000   

       capital_gain  capital_loss  hours_per_week  native_country  \
count  22793.000000  22793.000000    22793.000000    22793.000000   
mean    1069.960821     90.008775       40.442987       36.680911   
std     7355.247635    409.823252       12.398171        7.912790   
min        0.000000      0.000000        1.000000        0.000000   
25%        0.000000      0.000000       40.000000       39.000000   
50%        0.000000      0.000000       40.000000       39.000000   
75%        0.000000      0.000000       45.000000       39.000000   
max    99999.000000   4356.000000       99.000000       41.000000   

             income  
count  22793.000000  
mean       0.239372  
std        0.426709  
min        0.000000  
25%        0.000000  
50%        0.000000  
75%        0.000000  
max        1.000000

In [16]:
# An insight into the test data set to see if any 
testSet.describe()

age    workclass        fnlwgt    education  education_num  \
count  9768.000000  9768.000000  9.768000e+03  9768.000000    9768.000000   
mean     38.572277     3.849713  1.890034e+05    10.278153      10.080364   
std      13.611915     1.469508  1.048279e+05     3.875101       2.561098   
min      17.000000     0.000000  1.228500e+04     0.000000       1.000000   
25%      28.000000     4.000000  1.172738e+05     9.000000       9.000000   
50%      37.000000     4.000000  1.781215e+05    11.000000      10.000000   
75%      48.000000     4.000000  2.377010e+05    12.000000      12.000000   
max      90.000000     8.000000  1.366120e+06    15.000000      16.000000   

       marital_status   occupation  relationship         race          sex  \
count     9768.000000  9768.000000   9768.000000  9768.000000  9768.000000   
mean         2.609746     6.547400      1.432023     3.675164     0.676290   
std          1.493316     4.204217      1.608404     0.839799     0.467915   
min          0.000000     0.000000      0.000000     0.000000     0.000000   
25%          2.000000     3.000000      0.000000     4.000000     0.000000   
50%          2.000000     7.000000      1.000000     4.000000     1.000000   
75%          4.000000    10.000000      3.000000     4.000000     1.000000   
max          6.000000    14.000000      5.000000     4.000000     1.000000   

       capital_gain  capital_loss  hours_per_week  native_country       income  
count   9768.000000   9768.000000     9768.000000     9768.000000  9768.000000  
mean    1095.588350     80.992015       40.424550       36.807432     0.244165  
std     7455.275726    386.418950       12.228827        7.611704     0.429613  
min        0.000000      0.000000        1.000000        0.000000     0.000000  
25%        0.000000      0.000000       40.000000       39.000000     0.000000  
50%        0.000000      0.000000       40.000000       39.000000     0.000000  
75%        0.000000      0.000000       45.000000       39.000000     0.000000  
max    99999.000000   3770.000000       99.000000       41.000000     1.000000

In [17]:
trainSet.shape

(22793, 15)

In [18]:
testSet.shape

(9768, 15)

In [20]:
yTest = list(trainSet.iloc[:,-1])

In [28]:
# Test the model on testing set
testPred = naiveBayesian(trainSet, testSet)
print('Accuracy of prediction for testing set:', accuracy(testPred, testSet))

Number of exact matches in predictions: 7824/9768
Mean Squared Error(MSE): 0.19901719901719903
Root Mean Squared Error (RMSE): 0.44611343738694875
Mean Absolute Error(MSE): 0.19901719901719903
Length of predictions: 9768
Length of actual values: 9768
Accuracy of prediction for testing set: 80.098
